In [1]:
from rlbench.environment import Environment
from rlbench.action_modes import ArmActionMode, ActionMode
from rlbench.observation_config import ObservationConfig
from rlbench.tasks import *
import numpy as np
from pyrep.const import ConfigurationPathAlgorithms as Algos
import scipy as sp
from quaternion import from_rotation_matrix, quaternion
from scipy.spatial.transform import Rotation

In [2]:
def skew(x):
    return np.array([[0, -x[2], x[1]],
                    [x[2], 0, -x[0]],
                    [-x[1], x[0], 0]])


def sample_normal_pose(pos_scale, rot_scale):
    '''
    Samples a 6D pose from a zero-mean isotropic normal distribution
    '''
    pos = np.random.normal(scale=pos_scale)
        
    eps = skew(np.random.normal(scale=rot_scale))
    R = sp.linalg.expm(eps)
    quat_wxyz = from_rotation_matrix(R)

    return pos, quat_wxyz


class RandomAgent:

    def act(self, obs):
        delta_pos = [(np.random.rand() * 2 - 1) * 0.005, 0, 0]
        delta_quat = [0, 0, 0, 1] # xyzw
        gripper_pos = [np.random.rand() > 0.5]
        return delta_pos + delta_quat + gripper_pos


class NoisyObjectPoseSensor:

    def __init__(self, env):
        self._env = env

        self._pos_scale = [0.005, 0.005, 0.005]
        self._rot_scale = [0.01] * 3

    def get_poses(self):
        objs = self._env._scene._active_task.get_base().get_objects_in_tree(exclude_base=True, first_generation_only=False)
        obj_poses = {}

        for obj in objs:
            name = obj.get_name()
            pose = obj.get_pose()

            pos, quat_wxyz = sample_normal_pose(self._pos_scale, self._rot_scale)
            gt_quat_wxyz = quaternion(pose[6], pose[3], pose[4], pose[5])
            perturbed_quat_wxyz = quat_wxyz * gt_quat_wxyz

            pose[:3] += pos
            pose[3:] = [perturbed_quat_wxyz.x, perturbed_quat_wxyz.y, perturbed_quat_wxyz.z, perturbed_quat_wxyz.w]

            obj_poses[name] = pose

        return obj_poses


In [3]:
def contains(r1, r2):
    #whether r2 is within r1
    return r1[0] < r2[0] < r2[1] < r1[1] and r1[2] < r2[2] < r2[3] < r1[3] and r1[4] < r2[4] < r2[5] < r1[5]

def get_edge_points(obj_bbox,obj_mat):
    bbox_faceedge=np.array([[obj_bbox[0],0,0],
    [obj_bbox[1],0,0],
    [0,obj_bbox[2],0],
    [0,obj_bbox[3],0],
    [0,0,obj_bbox[4]],
    [0,0,obj_bbox[5]]]).T #3x6
    bbox_faceedge=np.vstack((bbox_faceedge,np.ones((1,6)))) #4x6
    box=(obj_mat@bbox_faceedge).T #6X3 face edge coords in world frame
    rect=min(box[:,0]),max(box[:,0]),min(box[:,1]),max(box[:,1]),min(box[:,2]),max(box[:,2]) #max/min along x,y,z world axes
    return rect

In [4]:
class StateMachine(object):
    def __init__(self):
        self.env=None
        self.home=None
        self.task=None
        self.sensor=None
        self.objs_dict=None
        
    def initialize(self):
        DATASET = ''
        obs_config = ObservationConfig()
        obs_config.set_all(True)
        obs_config.left_shoulder_camera.rgb = True
        obs_config.right_shoulder_camera.rgb = True
        action_mode = ActionMode(ArmActionMode.ABS_JOINT_POSITION)
        self.env = Environment(
            action_mode, DATASET, obs_config, False)
        self.sensor = NoisyObjectPoseSensor(self.env)
        self.env.launch()
        self.task = self.env.get_task(PutGroceriesInCupboard)
        self.task.reset()
        self.home = self.env._scene.get_observation().gripper_pose
        self.home[2]+=0.3
        # demos = task.get_demos(3, live_demos=live_demos)
        
    def get_objects(self):
        objs = self.env._scene._active_task.get_base().get_objects_in_tree(exclude_base=True, first_generation_only=False)
        objs_dict = {}
        for obj in objs:
            name = obj.get_name()
            pose = obj.get_pose()
            objs_dict[name] = obj
        return objs_dict

    # Move above object
    def move_to(self, pose, pad=0.05, ig=True):
        print(ig)
        target_pose = np.copy(pose)
        obs = self.env._scene.get_observation()
        init_pose=obs.gripper_pose
        obs = self.env._scene.get_observation()
        init_pose=obs.gripper_pose
        target_pose[2]+=pad
        path=self.env._robot.arm.get_path(np.array(target_pose[0:3]),quaternion=np.array([0,1,0,0]), trials=1000,ignore_collisions=ig, algorithm=Algos.RRTConnect)
        # TODO catch errors and deal with situations when path not found
        return path
    
    def grasp(self,obj):
        # TODO get feedback to check if grasp is successful
        cond=False
        while not cond:
            cond=self.env._robot.gripper.actuate(0,2)
            self.env._scene.step()
            print("Grasping")
        cond = self.env._robot.gripper.grasp(obj)
        return cond
    
    def release(self, obj):
        cond=False
        while not cond:
            cond=self.env._robot.gripper.actuate(1,0.1)
            self.env._scene.step()
        self.env._robot.gripper.release()
    
    def execute(self, path):
        done=False
        path.set_to_start()
        while not done:
            done = path.step()
            a = path.visualize()
            self.env._scene.step()
        return done
    
    def reset(self):
        self.task.reset()
    
    
    def is_within(self,obj1,obj2):
        #whether obj2 is within obj1
        obj1_bbox=obj1.get_bounding_box();obj1_mat=np.array(obj1.get_matrix()).reshape(3,4);
        obj2_bbox=obj2.get_bounding_box();obj2_mat=np.array(obj2.get_matrix()).reshape(3,4);
        obj1_rect= get_edge_points(obj1_bbox,obj1_mat)
        obj2_rect= get_edge_points(obj2_bbox,obj2_mat)
        return contains(obj1_rect,obj2_rect)
    
    def picking_bin_empty(self):
        '''
         Returns whether the picking bin is empty
        '''
        self.objs_dict=machine.get_objects()
        bin_obj=self.objs_dict['large_container'] #?Which one
        for obj_name in self.objs_dict.keys():
            if (not ('container' in obj_name)):
                if self.is_within(bin_obj,self.objs_dict[obj_name]):
                    return False
        return True
    def placing_bin_full(self):
        '''
         Returns whether the placing bin is full
        '''
        self.objs_dict=machine.get_objects()
        bin_obj=self.objs_dict['small_container1'] #?Which one
        for obj_name in self.objs_dict.keys():
            if (not ('container' in obj_name)):
                if not (self.is_within(bin_obj,self.objs_dict[obj_name])):
                    return False
        return True
    

In [5]:
machine = StateMachine()
machine.initialize()

/home/arcot/.local/lib/python3.7/site-packages/pyrep/pyrep.py:209: UserWarning: Could not change simulation timestep. You may need to change it to "custom dt" using simulation settings dialog.
  warnings.warn('Could not change simulation timestep. You may need '


In [67]:
machine.reset()
objects = ['soup_grasp_point']
for shape in objects:
    # move above object
    objs=machine.get_objects()
    objs_poses=machine.sensor.get_poses()
    # pose=objs[shape].get_pose()
    pose=objs_poses[shape]
    path=machine.move_to(pose,False)
    machine.execute(path)
    # grasp the object

True


In [54]:
g = machine.env._robot.gripper

In [155]:
g.get_joint_forces()

[-0.08085033297538757, 0.08105567842721939]

In [25]:
machine.grasp(objs["soup"])

Grasping
Grasping
Grasping
Grasping
Grasping
Grasping
Grasping


True

In [26]:
g.get_joint_forces()

[27.06987953186035, 23.386653900146484]

In [72]:
pose = machine.env._scene.get_observation().gripper_pose
pose[2]+=0.1
path = machine.move_to(pose,False)

True


In [73]:
done=False
path.set_to_start()
while not done:
    done = path.step()
    a = path.visualize()
    machine.env._scene.step()
    print(g.get_joint_forces())

[-0.03523921221494675, 0.033982131630182266]
[0.03376701474189758, -0.028164392337203026]
[-0.014067116193473339, 0.016644872725009918]
[-0.005408613942563534, 0.0027383356355130672]
[0.014031345956027508, -0.011700796894729137]
[0.06258098781108856, -0.048967793583869934]
[-0.40860748291015625, 0.18621185421943665]
[-0.12364418804645538, 0.03146675229072571]
[-0.2970965504646301, 0.10477574914693832]
[-0.18093954026699066, 0.05479971319437027]
[-0.19392713904380798, 0.09751023352146149]


In [ ]:
path_ = path._path_points.reshape(-1,path._num_joints)
a = np.zeros((path_.shape[0],1))
points = np.hstack([path_,a])

In [ ]:
points.shape

In [ ]:
for i in range(len(points)):
    machine.task.step(points[i,:])
    print(g.get_joint_forces())
    machine.env._scene.step()

In [17]:
g.get_joint_forces()

[10.181020736694336, 10.585058212280273]

In [46]:
g = machine.env._robot.gripper

In [24]:
g.get_joint_forces()

[-0.020189102739095688, 0.020785421133041382]

In [55]:
path._path_points = path_.flatten()

In [56]:
done=False
while not done:
    done = path.step()
#     machine.task.step([0,0,0,0,0,0,1,0])
    a = path.visualize()
    machine.env._scene.step()

In [6]:
path

NameError: name 'path' is not defined

In [27]:
g = machine.env._robot.gripper

In [28]:
g.get_joint_velocities()

[0.002987384796142578, 0.0021278858184814453]

In [35]:
g.set_joint_target_positions([0,0])

In [31]:
g.get_joint_target_positions()

[0.0, 0.0]

In [ ]:
g.set_joint_forces([30,30])

In [ ]:
machine.env._scene.step()

In [7]:
path

NameError: name 'path' is not defined

In [25]:
objs

{'chocolate_jello': <pyrep.objects.shape.Shape at 0x7f78a7748390>,
 'strawberry_jello': <pyrep.objects.shape.Shape at 0x7f78a7748588>,
 'soup': <pyrep.objects.shape.Shape at 0x7f78a7748198>,
 'tuna': <pyrep.objects.shape.Shape at 0x7f78a7748278>,
 'spam': <pyrep.objects.shape.Shape at 0x7f78a7748358>,
 'sugar': <pyrep.objects.shape.Shape at 0x7f78a7748400>,
 'coffee': <pyrep.objects.shape.Shape at 0x7f78a77485c0>,
 'crackers': <pyrep.objects.shape.Shape at 0x7f78a77484e0>,
 'mustard': <pyrep.objects.shape.Shape at 0x7f78d80efc88>,
 'waypoint1': <pyrep.objects.dummy.Dummy at 0x7f78a7748748>,
 'boundary_root': <pyrep.objects.shape.Shape at 0x7f78d80efba8>,
 'chocolate_jello_visual': <pyrep.objects.shape.Shape at 0x7f78d80efbe0>,
 'chocolate_jello_grasp_point': <pyrep.objects.dummy.Dummy at 0x7f78d80efef0>,
 'strawberry_jello_visual': <pyrep.objects.shape.Shape at 0x7f78d80efb00>,
 'strawberry_jello_grasp_point': <pyrep.objects.dummy.Dummy at 0x7f78d80efe10>,
 'soup_visual': <pyrep.object

In [32]:
a =  objs["chocolate_jello"].get_parent()

In [33]:
a.get_name()

'Panda_gripper_attachPoint'

In [35]:
objs["soup"].get_parent().get_name()

'put_groceries_in_cupboard'

In [3]:
machine.env._pyrep.get_objects_in_tree(g)[0]

NameError: name 'machine' is not defined

In [73]:
obj_list = machine.env._pyrep.get_objects_in_tree(g)

In [74]:
[i.get_name() for i in obj_list]

['Panda_gripper_joint1',
 'Panda_gripper_joint2',
 'Panda_gripper_visual',
 'Panda_gripper_attachPoint',
 'Panda_tip',
 'Panda_gripper_attachProxSensor',
 'cam_wrist',
 'cam_wrist_mask',
 'Panda_leftfinger_respondable',
 'Panda_rightfinger_respondable',
 'Panda_leftfinger_visible',
 'Panda_gripper_touchSensor1',
 'Panda_rightfinger_visual',
 'Panda_gripper_touchSensor0',
 'Panda_leftfinger_force_contact',
 'Panda_rightfinger_force_contact']

In [65]:
g.actuate(1,1)

True

In [72]:
machine.env._scene.step()

In [69]:
g.release()

In [77]:
objs["soup"].get_bullet_friction()

0.5

In [2]:
machine.env

NameError: name 'machine' is not defined